<!--
#  Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#    Licensed under the Apache License, Version 2.0 (the "License").
#    You may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.
-->

# Creating EBS Volume for Teams to use
## Content
1. Admin Operations
    1. [Parameters](#Parameters)
    2. [Cleanup](#Cleanup)
    3. [Creating the EBS Volume](#Creating-the-EBS-Volume)
    4. [Creating the K8 Volume](#Creating-the-K8-Volume)
2. User Operations
    1. [Creating the K8 Volume Claim](#Creating-the-K8-Volume-Claim)
    2. [Creating the Profile with the required AZ](#Creating-the-Profile-with-the-required-AZ)
    3. [Running the container](#Running-the-container)
---
---





In [ ]:
from aws_orbit_sdk import controller
from aws_orbit_sdk.magics.orbit import OrbitWorkbenchMagics 
import json
import boto3
from aws_orbit_sdk.common import get_workspace

In [ ]:
# we will need the team kms key from workspace
workspace = get_workspace()
team_kms_key = workspace['TeamKmsKeyArn']
image = workspace['BaseImageAddress']

In [ ]:
%cd ebs

In [ ]:
env_name = %env AWS_ORBIT_ENV
team_name = %env AWS_ORBIT_TEAM_SPACE
(env_name,team_name)

## Parameters

In [ ]:
pv_name = 'my-pv1'
pvc_name = 'my-pvc1'
az = 'us-west-2a'
volume_size = 20 #gb

## Cleanup

In [ ]:
!kubectl delete pvc $pvc_name --force

In [ ]:
!kubectl delete pv $pv_name --force

## Creating the EBS Volume

In [ ]:
!echo aws ec2 create-volume --availability-zone=$az --encrypted \
           --size=$volume_size --volume-type=gp2 --kms-key-id $team_kms_key

In [ ]:
res = !aws ec2 create-volume --availability-zone=$az --encrypted \
           --size=$volume_size --volume-type=gp2 
res

In [ ]:
ebs_vol = json.loads('\n'.join(res))
ebs_vol

In [ ]:
volume_id = ebs_vol['VolumeId']
volume_id

In [ ]:
!aws ec2 wait volume-available --volume-ids $volume_id

## Creating the K8 Volume

In [ ]:
with open("pv.yaml", "w") as file:
    file.write("""
apiVersion: v1
kind: PersistentVolume
metadata:
  name: {pv_name}
  labels:
    type: {pv_name}
spec:
  storageClassName: ebs-{team_name}-gp2  
  capacity:
    storage: 5Gi
  accessModes:
    - ReadWriteOnce
  awsElasticBlockStore:
    volumeID: {volume_id}
    fsType: xfs        
    """.format(team_name=team_name,pv_name=pv_name,volume_id=volume_id)
)




In [ ]:
!cat pv.yaml

In [ ]:
!kubectl apply -f pv.yaml

## User Section

## Creating the K8 Volume Claim

In [ ]:
with open("pvc.yaml", "w") as file:
    file.write("""
kind: PersistentVolumeClaim
apiVersion: v1
metadata:
    name: {pvc_name}
    labels:
        type: {pvc_name}
spec:
    accessModes:
    - ReadWriteOnce
    storageClassName: ebs-{team_name}-gp2  
    resources:
        requests:
            storage: 5Gi
    selector:
        matchLabels:
            type: {pv_name} 
    """.format(team_name=team_name,pv_name=pv_name,pvc_name=pvc_name)
)




In [ ]:
!cat pvc.yaml

In [ ]:
!kubectl apply -f pvc.yaml

## Creating the PodSetting with the required AZ

In [ ]:
import json

customname = "orbit-custom-volumes-"+team_name

with open("podsetting_ebs.yaml", "w") as file:
    file.write("""
    kind: PodSetting
    apiVersion: orbit.aws/v1
    metadata:
      labels:
        orbit/env: {env_name}
        orbit/space: team
        orbit/team: {team_name}
      name: {customname}
      namespace: {team_name}
    spec:
      affinity:
        nodeAffinity:
          preferredDuringSchedulingIgnoredDuringExecution:
            - preference:
                matchExpressions:
                  - key: topology.kubernetes.io/zone
                    operator: In
                    values:
                    - {az}
              weight: 1
      containerSelector:
        jsonpath: metadata.labels.app
      desc: Example EFS orbit-{customname}
      env:
        - name: custom_name
          value: custom_value
      image: >-
        {image}
      podSelector:
        matchExpressions:
          - key: orbit/{customname}
            operator: Exists
      resources:
        limits:
          cpu: '1.0'
          memory: 1Gi
        requests:
          cpu: '1.0'
          memory: 1Gi
      securityContext:
        runAsUser: 1000
      volumeMounts:
        - mountPath: /ebs
          name: ebs-volume
      volumes:
        - name: ebs-volume
          persistentVolumeClaim:
            claimName: {pvc_name}
    """.format(team_name=team_name,env_name=env_name,pvc_name=pvc_name,customname=customname,image=image,az=az)
)



In [ ]:
!kubectl apply -f podsetting_ebs.yaml -n {team_name}

## Running the container

In [ ]:
run = {    
      "tasks":  [
            {
                  "notebookName": "test-ebs.ipynb",
                  "sourcePath": "shared/samples/notebooks/M-Admin/ebs",
                  "targetPath": "shared/regression/notebooks/M-Admin/ebs",
                  "params": {
                        "test" : "1"
                  }      
            }
      ],
      "compute": {
          "container" : {
              "p_concurrent": "1"
          },
          "node_type": "ec2",
          "podsetting":customname,
          "labels": {
            "my-jobid": "1"
          }          
      }
}

with open("run.json", 'w') as f:
    json.dump(run, f)

In [ ]:
%%time

!orbit run notebook --env $env_name --team $team_name --user testing --wait --tail-logs run.json


## Cleanup

In [ ]:
# Using our label to delete the job
!kubectl delete job -l my-jobid=1

In [ ]:
!kubectl delete podsetting -n {team_name} {customname}

In [ ]:
!kubectl delete pvc $pvc_name --force

In [ ]:
!kubectl delete pv $pv_name --force

In [ ]:
!aws ec2 delete-volume --volume-id $volume_id